# Домашнее задание 4

## Обучение самописной CNN

In [18]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import Module, Sequential, Conv2d, ReLU, MaxPool2d, Linear, Dropout, CrossEntropyLoss
from torch.optim import Adam
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR100
from torchvision import models
from tqdm import trange

In [2]:
train_dataset = CIFAR100(root='./dataset/CIFAR100',
                        train=True,
                        transform=transforms.Compose(
                             [transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
                        download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True)
test_dataset = CIFAR100(root='./dataset/CIFAR100', train=False,
                       download=True, transform=transforms.Compose(
                           [transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100,
                                          shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
class ConvNet(Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = Sequential(Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
                                    ReLU(), MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = Sequential(Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
                                    ReLU(), MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = Dropout()
        self.fc1 = Linear(4096, 1000)
        self.fc2 = Linear(1000, 100)
    
    def forward(self, x): 
     out = self.layer1(x) 
     out = self.layer2(out) 
     out = out.reshape(out.size(0), -1) 
     out = self.drop_out(out) 
     out = self.fc1(out) 
     out = self.fc2(out) 
     return out


In [15]:
num_epochs = 50 
num_classes = 10 
batch_size = 100 
learning_rate = 0.001

In [16]:
model = ConvNet().cuda()
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)

In [17]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.cuda(), labels.cuda()
        # Прямой запуск
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/50], Step [100/500], Loss: 3.7171, Accuracy: 14.00%
Epoch [1/50], Step [200/500], Loss: 3.7248, Accuracy: 17.00%
Epoch [1/50], Step [300/500], Loss: 3.2708, Accuracy: 26.00%
Epoch [1/50], Step [400/500], Loss: 3.1999, Accuracy: 17.00%
Epoch [1/50], Step [500/500], Loss: 3.0774, Accuracy: 27.00%
Epoch [2/50], Step [100/500], Loss: 3.0184, Accuracy: 21.00%
Epoch [2/50], Step [200/500], Loss: 2.8530, Accuracy: 27.00%
Epoch [2/50], Step [300/500], Loss: 2.9501, Accuracy: 30.00%
Epoch [2/50], Step [400/500], Loss: 2.7764, Accuracy: 30.00%
Epoch [2/50], Step [500/500], Loss: 2.5389, Accuracy: 39.00%
Epoch [3/50], Step [100/500], Loss: 2.7045, Accuracy: 27.00%
Epoch [3/50], Step [200/500], Loss: 2.7915, Accuracy: 34.00%
Epoch [3/50], Step [300/500], Loss: 2.8541, Accuracy: 27.00%
Epoch [3/50], Step [400/500], Loss: 2.8618, Accuracy: 29.00%
Epoch [3/50], Step [500/500], Loss: 2.4274, Accuracy: 35.00%
Epoch [4/50], Step [100/500], Loss: 2.6414, Accuracy: 36.00%
Epoch [4/50], Step [200/

In [23]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images.cuda()).cpu()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy of the model on the 10000 test images: {(correct / total) * 100}')


Test Accuracy of the model on the 10000 test images: 40.29


In [27]:

torch.save(model.state_dict(), './Model/conv_cifar100.ckpt')

## Обучение ResNet50

In [30]:
resnet50 = models.resnet50(pretrained=True).cuda()

g:\Scooling\Praktikum\3Chetvert\Torch\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\Scooling\Praktikum\3Chetvert\Torch\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [31]:
num_epochs = 10
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.cuda(), labels.cuda()
        # Прямой запуск
        outputs = resnet50(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/10], Step [100/500], Loss: 12.8718, Accuracy: 1.00%
Epoch [1/10], Step [200/500], Loss: 12.8600, Accuracy: 0.00%
Epoch [1/10], Step [300/500], Loss: 13.4766, Accuracy: 0.00%
Epoch [1/10], Step [400/500], Loss: 12.6906, Accuracy: 0.00%
Epoch [1/10], Step [500/500], Loss: 12.3689, Accuracy: 0.00%
Epoch [2/10], Step [100/500], Loss: 14.1139, Accuracy: 0.00%
Epoch [2/10], Step [200/500], Loss: 13.4952, Accuracy: 0.00%
Epoch [2/10], Step [300/500], Loss: 12.6936, Accuracy: 0.00%
Epoch [2/10], Step [400/500], Loss: 12.8825, Accuracy: 0.00%
Epoch [2/10], Step [500/500], Loss: 13.3141, Accuracy: 0.00%
Epoch [3/10], Step [100/500], Loss: 12.5405, Accuracy: 0.00%
Epoch [3/10], Step [200/500], Loss: 13.1989, Accuracy: 0.00%
Epoch [3/10], Step [300/500], Loss: 13.6035, Accuracy: 0.00%
Epoch [3/10], Step [400/500], Loss: 13.2802, Accuracy: 0.00%
Epoch [3/10], Step [500/500], Loss: 13.7321, Accuracy: 0.00%
Epoch [4/10], Step [100/500], Loss: 12.7920, Accuracy: 0.00%
Epoch [4/10], Step [200/